# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# ! echo "4/sAFT6Lt4k6urQdvj0H6vghJt7kl6X9dCG0gd-XY_1mMDstx7QiAJ0qM" | python -c "from google.colab import drive; drive.mount('/content/drive')"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "af"
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
gdrive_path = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)
os.environ["gdrive_path"] = gdrive_path
!mkdir -p "$gdrive_path"
# !mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"

In [3]:
!echo $gdrive_path
# !ls $gdrive_path

/content/drive/My Drive/masakhane/en-af-baseline


In [4]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 51kB 2.0MB/s 


In [5]:
# change working directory
os.chdir(gdrive_path)

# Download our corpus
! if ! (( test -f jw300.$src ) && ( test -f jw300.$tgt )); then opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q; else echo "Opus files are already present, skipping download."; fi

# extract the corpus file
! if test -f JW300_latest_xml_$src-$tgt.xml.gz; then gunzip JW300_latest_xml_$src-$tgt.xml.gz; elif test -f JW300_latest_xml_$tgt-$src.xml.gz; then gunzip JW300_latest_xml_$tgt-$src.xml.gz; else echo "ERROR: missing corpus file!"; fi

Opus files are already present, skipping download.
ERROR: missing corpus file!


In [6]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
with open(source_file) as f:
  for _, line in enumerate(f):
    source.append(line)
with open(target_file) as f:
  for _, line in enumerate(f):
    target.append(line)

df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
df.head(3)

,source_sentence,target_sentence
0,‘ They Lifted Me Out of Deep Depression ’\n,‘ Hulle het my uit diep depressie gehelp ’\n
1,A woman from England wrote :\n,’ n Vrou van Engeland het geskryf :\n
2,“ Dear Sir :\n,“ Geagte Meneer :\n


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [7]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [8]:
# This section does the split between train/test/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and 1000 test set. In practice, it's useful to use an external test set

# Do the split between dev/test/train and create parallel corpora
num_dev_patterns = 1000
num_test_patterns = 1000

# Lower case the corpora
df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

devtest = df_pp.tail(num_dev_patterns + num_test_patterns)
test = devtest.tail(num_test_patterns) # Herman
dev = devtest.head(num_dev_patterns)  # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns + num_test_patterns).index)

stripped[["source_sentence"]].to_csv("train.en", index=False)
stripped[["target_sentence"]].to_csv("train.af", index=False)

dev[["source_sentence"]].to_csv("dev.en", index=False)
dev[["target_sentence"]].to_csv("dev.af", index=False)

test[["source_sentence"]].to_csv("test.en", index=False)
test[["target_sentence"]].to_csv("test.af", index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  




---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [9]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git

# swap to branch that has the "ElanScheduler"! and install that :D
# ! cd joeynmt; git checkout scheduler; git pull --all; pip3 install . # pip install --upgrade --force-reinstall --no-deps <package>
! cd joeynmt; git checkout symlink; git pull --all; pip3 install . # pip install --upgrade --force-reinstall --no-deps <package>

# perform default installation
# ! cd joeynmt; git checkout master; git pull --all; pip3 install .

fatal: destination path 'joeynmt' already exists and is not an empty directory.
M	scripts/generate_copy_task.py
M	scripts/generate_reverse_task.py
M	scripts/get_iwslt14_bpe.sh
M	scripts/get_iwslt15_envi.sh
M	scripts/plot_validations.py
Already on 'symlink'
Your branch is up to date with 'origin/symlink'.
Fetching origin
Already up to date.
Processing /content/drive/My Drive/masakhane/en-af-baseline/joeynmt
     |████████████████████████████████| 266kB 7.5MB/s 
     |████████████████████████████████| 307kB 39.8MB/s 
     |████████████████████████████████| 51kB 21.8MB/s 
     |████████████████████████████████| 215kB 41.4MB/s 
     |████████████████████████████████| 737kB 41.6MB/s 
     |████████████████████████████████| 51kB 13.9MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=69462 sha256=2eb100ac13e868b7e68943631f0ba842d5d081a3f1dee88932c5fd0189fd4217
  Stored in directory: /tmp/pip-ephem-wheel-cache-8heq9clf/wheels/4c/ba/65/529ea2efaa773ebbedd9df75a2058

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [30]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
# from os import path

os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman!

! if ! (( test -f vocab.$src ) && ( test -f vocab.$tgt )); then subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt; else echo "BPE vocab files already present..."; fi

! if ! test -f train.bpe.$src; then subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src; else echo "train.bpe.$src already present..."; fi
! if ! test -f train.bpe.$tgt; then subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt; else echo "train.bpe.$tgt already present..."; fi

! if ! test -f dev.bpe.$src; then subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src; else echo "dev.bpe.$src already present..."; fi
! if ! test -f dev.bpe.$tgt; then subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt; else echo "dev.bpe.$tgt already present..."; fi

! if ! test -f test.bpe.$src; then subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src; else echo "test.bpe.$src already present..."; fi
! if ! test -f test.bpe.$tgt; then subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt; else echo "test.bpe.$tgt already present..."; fi

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! if ! test $(ls -l $data_path | grep train | wc -l) -gt 0; then cp train.* $data_path; else echo "train files already in data directory..."; fi
! if ! test $(ls -l $data_path | grep test | wc -l) -gt 0; then cp test.* $data_path; else echo "test files already in data directory..."; fi
! if ! test $(ls -l $data_path | grep dev | wc -l) -gt 0; then cp dev.* $data_path; else echo "dev files already in data directory..."; fi
! if ! test -f $data_path/bpe.codes.4000; then cp bpe.codes.4000 $data_path ; else echo "bpe.codes.4000 already in data directory..."; fi
! ls $data_path

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! if ! test -f joeynmt/data/$src$tgt/vocab.txt; then joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt ; else echo "vocab.txt already in data directory..."; fi

# Some output
! echo "BPE Afrikaans Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt

BPE vocab files already present...
train.bpe.en already present...
train.bpe.af already present...
dev.bpe.en already present...
dev.bpe.af already present...
test.bpe.en already present...
test.bpe.af already present...
train files already in data directory...
test files already in data directory...
dev files already in data directory...
bpe.codes.4000 already in data directory...
bpe.codes.4000	dev.bpe.en  test.bpe.af  train.af      train.en
dev.af		dev.en	    test.bpe.en  train.bpe.af  vocab.txt
dev.bpe.af	test.af     test.en	 train.bpe.en
vocab.txt already in data directory...
BPE Afrikaans Sentences
"
"maar ons kan daarvan seker wees dat ons eer@@ likheid en ander goeie eienskappe vir ons hemelse vader baie ko@@ sb@@ aar@@ der is as enige ed@@ el@@ ste@@ en !
"
"as ons eer@@ lik is , het ons ’ n sk@@ oon gewe@@ te en vryheid van spraak in die bediening
"
Combined BPE Vocab
·@@
̣@@
̈@@
§
ז@@
inst
˜
;@@
ḥ
saja


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
# # This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# # (You can of course play with all the parameters if you'd like!)

# name = '%s%s' % (source_language, target_language)
# gdrive_path = os.environ["gdrive_path"]

# # Create the config
# config = """
# name: "{name}_transformer"

# data:
#     src: "{source_language}"
#     trg: "{target_language}"
#     train: "data/{name}/train.bpe"
#     dev:   "data/{name}/dev.bpe"
#     test:  "data/{name}/test.bpe"
#     level: "bpe"
#     lowercase: False
#     max_sent_length: 100
#     src_vocab: "data/{name}/vocab.txt"
#     trg_vocab: "data/{name}/vocab.txt"

# testing:
#     beam_size: 5
#     alpha: 1.0

# training:
#     #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
#     random_seed: 42
#     optimizer: "adam"
#     normalization: "tokens"
#     adam_betas: [0.9, 0.999] 
#     scheduling: "elan"            # Try switching to Elan scheduling
#     learning_rate_decay_length: 5000 # number of steps to reduce by the decay factor for Elan method
#     learning_rate_peak: 0.002  # peak for Elan scheduler (default: 1)
#     learning_rate_warmup: 2000  # warmup steps for Elan scheduler
#     learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
#     learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
#     patience: 8
#     decrease_factor: 0.7
#     loss: "crossentropy"
#     learning_rate: 0.0002
#     learning_rate_min: 0.00000001
#     weight_decay: 0.0
#     label_smoothing: 0.1
#     batch_size: 4096
#     batch_type: "token"
#     eval_batch_size: 3600
#     eval_batch_type: "token"
#     batch_multiplier: 1
#     early_stopping_metric: "ppl"
#     epochs: 30 # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
#     validation_freq: 500 # 4000 # Decrease this for testing
#     logging_freq: 100
#     eval_metric: "bleu"
#     model_dir: "models/{name}_transformer"
#     overwrite: True
#     shuffle: True
#     use_cuda: True
#     max_output_length: 100
#     print_valid_sents: [0, 1, 2, 3]
#     keep_last_ckpts: 3

# model:
#     initializer: "xavier"
#     bias_initializer: "zeros"
#     init_gain: 1.0
#     embed_initializer: "xavier"
#     embed_init_gain: 1.0
#     tied_embeddings: True
#     tied_softmax: True
#     encoder:
#         type: "transformer"
#         num_layers: 3
#         num_heads: 8
#         embeddings:
#             embedding_dim: 512
#             scale: True
#             dropout: 0.
#         # typically ff_size = 4 x hidden_size
#         hidden_size: 512
#         ff_size: 2048
#         dropout: 0.3
#     decoder:
#         type: "transformer"
#         num_layers: 3
#         num_heads: 8
#         embeddings:
#             embedding_dim: 512
#             scale: True
#             dropout: 0.
#         # typically ff_size = 4 x hidden_size
#         hidden_size: 512
#         ff_size: 2048
#         dropout: 0.25
# """.format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
# with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
#     f.write(config)

In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    #load_model: "{gdrive_path}/joeynmt/models/{name}_transformer/3500.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    load_model: "{gdrive_path}/Backup_models/Copy of best.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "noam"            # Try switching to Elan scheduling
    learning_rate_decay_length: 5000 # number of steps to reduce by the decay factor for Elan method
    learning_rate_peak: 0.002  # peak for Elan scheduler (default: 1)
    learning_rate_warmup: 2000  # warmup steps for Elan scheduler
    learning_rate_factor: 1       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    patience: 8
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0002
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30 # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500 # 4000 # Decrease this for testing
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: True
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 3
        num_heads: 8
        embeddings:
            embedding_dim: 512
            scale: True
            dropout: 0.
        # typically ff_size = 4 x hidden_size
        hidden_size: 512
        ff_size: 2048
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 3
        num_heads: 8
        embeddings:
            embedding_dim: 512
            scale: True
            dropout: 0.
        # typically ff_size = 4 x hidden_size
        hidden_size: 512
        ff_size: 2048
        dropout: 0.25
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open(os.path.join(gdrive_path, f"joeynmt/configs/transformer_{name}.yaml"),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [0]:
# # Train the model
# # You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 

# os.chdir(os.path.join(gdrive_path, "joeynmt"))

# ! python3 joeynmt/__main__.py train configs/transformer_$src$tgt.yaml
# # !cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

In [0]:
# Copy the created models from the notebook storage to google drive for persistant storage 
# !cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"
! mv "$gdrive_path/../../masakhane_models_backup/models" "$gdrive_path/joeynmt"

In [19]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 500	Loss: 90178.07812	PPL: 37.62176	bleu: 0.90432	LR: 0.00069877	*
Steps: 1000	Loss: 73574.21094	PPL: 19.29143	bleu: 2.83208	LR: 0.00139754	*
Steps: 1500	Loss: 67591.24219	PPL: 15.16492	bleu: 4.42870	LR: 0.00114109	*
Steps: 2000	Loss: 63030.15234	PPL: 12.62282	bleu: 5.49023	LR: 0.00098821	*
Steps: 2500	Loss: 58729.75781	PPL: 10.61762	bleu: 7.85880	LR: 0.00088388	*
Steps: 3000	Loss: 54868.16016	PPL: 9.09000	bleu: 10.19263	LR: 0.00080687	*
Steps: 3500	Loss: 51913.49219	PPL: 8.07133	bleu: 11.93531	LR: 0.00074702	*
Steps: 4000	Loss: 48699.18750	PPL: 7.09235	bleu: 15.10315	LR: 0.00069877	*
Steps: 4500	Loss: 46169.38281	PPL: 6.40610	bleu: 17.91368	LR: 0.00065881	*
Steps: 5000	Loss: 43576.91016	PPL: 5.77168	bleu: 21.04279	LR: 0.00062500	*
Steps: 5500	Loss: 40994.23438	PPL: 5.20214	bleu: 23.94864	LR: 0.00059591	*
Steps: 6000	Loss: 38960.73438	PPL: 4.79354	bleu: 26.45850	LR: 0.00057054	*
Steps: 6500	Loss: 36407.32422	PPL: 4.32561	bleu: 30.21616	LR: 0.00054816	*
Steps: 7000	Loss: 34931.22

In [22]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2019-10-16 13:50:44,139 -  dev bleu:  45.10 [Beam search decoding with beam size = 5 and alpha = 1.0]
2019-10-16 13:51:41,319 - test bleu:  45.48 [Beam search decoding with beam size = 5 and alpha = 1.0]
